# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [134]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [135]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [136]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [137]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [138]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [139]:
your_name = 'Anurag Kumar Singh' # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [140]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [141]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,region,charges
27,55,female,36.0525,2,no,northwest,14354.299732
752,64,male,41.6955,0,no,northwest,16626.327062
1258,55,male,41.4865,3,no,northwest,35174.389244
384,44,male,24.3485,2,no,northeast,9713.966711
406,33,female,26.7410,0,no,southeast,4896.564543


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [142]:
num_rows = dataframe.shape[0]
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [143]:
num_cols = dataframe.shape[1]
print(num_cols)

7


**Q: What are the column titles of the input variables?**

In [144]:
input_cols = dataframe.columns[0:-1]
input_cols

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region'], dtype='object')

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [145]:
categorical_cols = []
for i in input_cols:
  if str(dataframe.dtypes[i]) != 'int64' and str(dataframe.dtypes[i]) != 'float64':
    categorical_cols.append(i)

categorical_cols

['sex', 'smoker', 'region']

**Q: What are the column titles of output/target variable(s)?**

In [146]:
output_cols = [dataframe.columns[-1]]
print(output_cols)

['charges']


**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [147]:
# Write your answer here

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [148]:
!pip install jovian --upgrade -q

In [149]:
import jovian

In [150]:
jovian.commit(project=project_name)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/anurag3301/02-insurance-linear-regression


'https://jovian.ai/anurag3301/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [151]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [152]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[55.    ,  0.    , 36.0525,  2.    ,  0.    ,  1.    ],
        [64.    ,  1.    , 41.6955,  0.    ,  0.    ,  1.    ],
        [55.    ,  1.    , 41.4865,  3.    ,  0.    ,  1.    ],
        ...,
        [56.    ,  1.    , 28.5285,  0.    ,  0.    ,  0.    ],
        [18.    ,  0.    , 33.3355,  0.    ,  0.    ,  0.    ],
        [43.    ,  1.    , 28.633 ,  0.    ,  0.    ,  0.    ]]),
 array([[14354.2997325],
        [16626.3270615],
        [35174.3892435],
        ...,
        [13063.5386505],
        [ 2578.3710615],
        [ 7999.721379 ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [153]:
inputs = torch.tensor(inputs_array, dtype=torch.float32)
targets = torch.tensor(targets_array, dtype=torch.float32)

In [154]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [155]:
dataset = TensorDataset(inputs, targets)
len(dataset)

1271

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [156]:
val_percent = 0.05 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, (train_size, val_size)) # Use the random_split function to split dataset into 2 parts of the desired length
len(train_ds), len(val_ds)

(1208, 63)

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [157]:
batch_size = 100

In [158]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [159]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[35.0000,  1.0000, 42.4600,  1.0000,  0.0000,  3.0000],
        [64.0000,  0.0000, 35.0075,  2.0000,  0.0000,  0.0000],
        [54.0000,  0.0000, 23.6170,  3.0000,  0.0000,  1.0000],
        [47.0000,  0.0000, 32.3070,  1.0000,  0.0000,  2.0000],
        [34.0000,  0.0000, 29.0510,  1.0000,  0.0000,  1.0000],
        [58.0000,  1.0000, 39.2700,  0.0000,  0.0000,  3.0000],
        [63.0000,  1.0000, 23.8260,  1.0000,  0.0000,  1.0000],
        [61.0000,  0.0000, 32.9120,  3.0000,  1.0000,  2.0000],
        [59.0000,  0.0000, 30.4920,  3.0000,  0.0000,  2.0000],
        [24.0000,  1.0000, 34.1715,  0.0000,  1.0000,  0.0000],
        [21.0000,  1.0000, 34.1220,  0.0000,  0.0000,  2.0000],
        [46.0000,  0.0000, 35.5300,  2.0000,  0.0000,  0.0000],
        [30.0000,  0.0000, 35.6400,  1.0000,  0.0000,  3.0000],
        [59.0000,  0.0000, 40.4415,  1.0000,  1.0000,  0.0000],
        [38.0000,  0.0000, 29.9915,  1.0000,  0.0000,  0.0000],
        [55.0000,  0.0000, 32.81

Let's save our work by committing to Jovian.

In [160]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/anurag3301/02-insurance-linear-regression


'https://jovian.ai/anurag3301/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [161]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [162]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.mse_loss(out, targets)                          # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.mse_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [163]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [164]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3201,  0.3886,  0.3905, -0.2959, -0.2323, -0.1969]],
        requires_grad=True), Parameter containing:
 tensor([-0.2601], requires_grad=True)]

One final commit before we train the model.

In [165]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/anurag3301/02-insurance-linear-regression


'https://jovian.ai/anurag3301/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [197]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [198]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 27569044.0}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [199]:
model = InsuranceModel()

In [200]:
epochs = 1000
lr = 1e-4
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 124562840.0000
Epoch [40], val_loss: 137630272.0000
Epoch [60], val_loss: 121388392.0000
Epoch [80], val_loss: 135193024.0000
Epoch [100], val_loss: 135985808.0000
Epoch [120], val_loss: 125477880.0000
Epoch [140], val_loss: 114577928.0000
Epoch [160], val_loss: 123990240.0000
Epoch [180], val_loss: 117851400.0000
Epoch [200], val_loss: 109612888.0000
Epoch [220], val_loss: 137076208.0000
Epoch [240], val_loss: 115094944.0000
Epoch [260], val_loss: 113507800.0000
Epoch [280], val_loss: 104494600.0000
Epoch [300], val_loss: 113768144.0000
Epoch [320], val_loss: 100894328.0000
Epoch [340], val_loss: 109321080.0000
Epoch [360], val_loss: 99041112.0000
Epoch [380], val_loss: 98300928.0000
Epoch [400], val_loss: 135058848.0000
Epoch [420], val_loss: 94775088.0000
Epoch [440], val_loss: 158186176.0000
Epoch [460], val_loss: 93196288.0000
Epoch [480], val_loss: 97866968.0000
Epoch [500], val_loss: 90031248.0000
Epoch [520], val_loss: 107297040.0000
Epoch [540], val_loss:

In [201]:
epochs = 10000
lr = 1e-4
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 69348760.0000
Epoch [40], val_loss: 69847480.0000
Epoch [60], val_loss: 107042536.0000
Epoch [80], val_loss: 66575576.0000
Epoch [100], val_loss: 73511480.0000
Epoch [120], val_loss: 76269808.0000
Epoch [140], val_loss: 75803928.0000
Epoch [160], val_loss: 68048256.0000
Epoch [180], val_loss: 62562164.0000
Epoch [200], val_loss: 62925484.0000
Epoch [220], val_loss: 67656624.0000
Epoch [240], val_loss: 64605664.0000
Epoch [260], val_loss: 63501724.0000
Epoch [280], val_loss: 61264220.0000
Epoch [300], val_loss: 109865720.0000
Epoch [320], val_loss: 59715920.0000
Epoch [340], val_loss: 58879748.0000
Epoch [360], val_loss: 58330024.0000
Epoch [380], val_loss: 61562736.0000
Epoch [400], val_loss: 57848568.0000
Epoch [420], val_loss: 58328976.0000
Epoch [440], val_loss: 57633052.0000
Epoch [460], val_loss: 56347120.0000
Epoch [480], val_loss: 63108628.0000
Epoch [500], val_loss: 63535520.0000
Epoch [520], val_loss: 60512320.0000
Epoch [540], val_loss: 55973788.0000
Epo

In [202]:
epochs = 10000
lr = 1e-4
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 27475198.0000
Epoch [40], val_loss: 27715510.0000
Epoch [60], val_loss: 31812526.0000
Epoch [80], val_loss: 27042096.0000
Epoch [100], val_loss: 27880630.0000
Epoch [120], val_loss: 39645920.0000
Epoch [140], val_loss: 29144032.0000
Epoch [160], val_loss: 28934934.0000
Epoch [180], val_loss: 27255940.0000
Epoch [200], val_loss: 27946870.0000
Epoch [220], val_loss: 27249662.0000
Epoch [240], val_loss: 28087186.0000
Epoch [260], val_loss: 27159958.0000
Epoch [280], val_loss: 27319594.0000
Epoch [300], val_loss: 27780888.0000
Epoch [320], val_loss: 26927382.0000
Epoch [340], val_loss: 27275074.0000
Epoch [360], val_loss: 27089540.0000
Epoch [380], val_loss: 27888580.0000
Epoch [400], val_loss: 28472438.0000
Epoch [420], val_loss: 32461296.0000
Epoch [440], val_loss: 27052532.0000
Epoch [460], val_loss: 27132238.0000
Epoch [480], val_loss: 27175502.0000
Epoch [500], val_loss: 65910452.0000
Epoch [520], val_loss: 30400368.0000
Epoch [540], val_loss: 28562156.0000
Epoch

In [203]:
epochs = 10000
lr = 1e-5
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 27141698.0000
Epoch [40], val_loss: 26445914.0000
Epoch [60], val_loss: 26845712.0000
Epoch [80], val_loss: 26598638.0000
Epoch [100], val_loss: 27251600.0000
Epoch [120], val_loss: 26463882.0000
Epoch [140], val_loss: 26937846.0000
Epoch [160], val_loss: 27119578.0000
Epoch [180], val_loss: 27087886.0000
Epoch [200], val_loss: 26658870.0000
Epoch [220], val_loss: 26564906.0000
Epoch [240], val_loss: 26734788.0000
Epoch [260], val_loss: 26735748.0000
Epoch [280], val_loss: 27575368.0000
Epoch [300], val_loss: 26720126.0000
Epoch [320], val_loss: 26615154.0000
Epoch [340], val_loss: 26904428.0000
Epoch [360], val_loss: 27308214.0000
Epoch [380], val_loss: 26977820.0000
Epoch [400], val_loss: 26365904.0000
Epoch [420], val_loss: 27287402.0000
Epoch [440], val_loss: 27629858.0000
Epoch [460], val_loss: 27056784.0000
Epoch [480], val_loss: 26989174.0000
Epoch [500], val_loss: 26480512.0000
Epoch [520], val_loss: 26469238.0000
Epoch [540], val_loss: 26856520.0000
Epoch

In [204]:
epochs = 10000
lr = 1e-5
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 26524632.0000
Epoch [40], val_loss: 26805314.0000
Epoch [60], val_loss: 27037492.0000
Epoch [80], val_loss: 26736470.0000
Epoch [100], val_loss: 26787620.0000
Epoch [120], val_loss: 26820050.0000
Epoch [140], val_loss: 26521192.0000
Epoch [160], val_loss: 26694474.0000
Epoch [180], val_loss: 27069182.0000
Epoch [200], val_loss: 27026488.0000
Epoch [220], val_loss: 26636188.0000
Epoch [240], val_loss: 26623968.0000
Epoch [260], val_loss: 26768768.0000
Epoch [280], val_loss: 26817294.0000
Epoch [300], val_loss: 26834006.0000
Epoch [320], val_loss: 27348368.0000
Epoch [340], val_loss: 26752306.0000
Epoch [360], val_loss: 27484464.0000
Epoch [380], val_loss: 27042514.0000
Epoch [400], val_loss: 26831924.0000
Epoch [420], val_loss: 26780720.0000
Epoch [440], val_loss: 26801964.0000
Epoch [460], val_loss: 26727010.0000
Epoch [480], val_loss: 27132208.0000
Epoch [500], val_loss: 26722114.0000
Epoch [520], val_loss: 26774216.0000
Epoch [540], val_loss: 26721390.0000
Epoch

**Q: What is the final validation loss of your model?**

In [208]:
val_loss = evaluate(model, val_loader)
val_loss

{'val_loss': 26793100.0}

Let's log the final validation loss to Jovian and commit the notebook

In [209]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [210]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/anurag3301/02-insurance-linear-regression


'https://jovian.ai/anurag3301/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [211]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [212]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([18.0000,  0.0000, 30.0080,  3.0000,  1.0000,  2.0000])
Target: tensor([21321.4375])
Prediction: tensor([31445.3047])


In [213]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([47.0000,  0.0000, 25.9600,  1.0000,  0.0000,  3.0000])
Target: tensor([9991.4150])
Prediction: tensor([9323.8350])


In [214]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([53.0000,  0.0000, 39.4900,  2.0000,  0.0000,  3.0000])
Target: tensor([13061.3750])
Prediction: tensor([15590.8486])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [184]:

jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/anurag3301/02-insurance-linear-regression
[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/anurag3301/02-insurance-linear-regression


'https://jovian.ai/anurag3301/02-insurance-linear-regression'